In [1]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!pip install spacy_transformers

!pip install -U spacy

In [ ]:
import spacy
import spacy_transformers
from spacy.tokens import DocBin
from tqdm import tqdm
import json

In [ ]:
!git clone https://github.com/stlyash/spacyResumeParcer.git

base config file is downloaded from https://spacy.io/usage/training/#quickstart

In [ ]:
cv_data = json.load(open('/content/spacyResumeParcer/data/training/train_data.json'))

In [ ]:
len(cv_data)

In [ ]:
!python -m spacy init fill-config /content/spacyResumeParcer/data/training/base_config.cfg /content/spacyResumeParcer/data/training/config.cfg

In [ ]:
from numpy import e
def get_spacy_doc (file, data):
  nlp = spacy.blank('en')
  db = DocBin()

  for text, annot in tqdm (data):
    doc = nlp.make_doc(text) 
    annot =  annot['entities']
    ents = []
    entity_indices = []
    for start, end, label in annot:
      skip_entity = False
      for idx in range(start,end):
        if idx in entity_indices:
          skip_entity = True
          break
      if skip_entity == True:
        continue

      entity_indices = entity_indices + list(range(start,end))

      try:
        span = doc.char_span(start, end, label = label, alignment_mode = 'strict')
      except:
        continue
      
      if span is None:
        err_data = str([start,end]) + "    " + str(text) + "\n"
      else:
        ents.append(span)

    try:
      doc.ents = ents
      db.add(doc)
    except:
      pass
      
  return db

In [ ]:
from sklearn.model_selection import train_test_split
train,test = train_test_split(cv_data,test_size = 0.3)
len(train),len(test)

In [ ]:
file  = open('error.txt','w')
db = get_spacy_doc(file,train)
db.to_disk('train_data.spacy')

db = get_spacy_doc(file,test)
db.to_disk('test_data.spacy')
file.close()

In [ ]:
!python -m spacy train /content/spacyResumeParcer/data/training/config.cfg --output ./output --paths.train ./train_data.spacy --paths.dev ./test_data.spacy --gpu-id 0

## Testing Model

In [ ]:
!pip install spacy-transformers

In [ ]:
nlp = spacy.load('/content/output/model-best')

In [ ]:
!pip install docx2txt
!pip install PyPDF2

In [ ]:
import docx2txt     # to convert docx files into text
import PyPDF2       # to convert pdf files into text

## Reading and Analyzing Bulk Resume(s)

In [ ]:
# Keywords to search in the resume
keyString = "C++,C,Python,Java,Excel,MatLab"
keylist = keyString.split(',')
keylen = len(keylist)

In [ ]:
import os  
import pandas as pd
import re

pathinp = r'/content/inputFolder'
pathparent = r'..'
os.chdir(pathinp)

fileList = os.listdir()

fileScore = []
degree = []
fileLen = len(fileList)
nameList = []
foundKeywordList = []
emailList = []
files = 0
indicator = 0
for fileName in fileList:

    # Printing percentage of files processed
    files+=1
    if files%(fileLen//4) == 0:
        indicator+=25
        print("Processed {}%".format(indicator))

    # Reading text from a file into an array
    keyfound = 0
    text=''
    try:
        resume = docx2txt.process(fileName)
        for i in resume:
                if i == '\n':
                    text += ' '
                else:
                    text+=i
    except:
        file = open(fileName,'rb')
        reader=PyPDF2.PdfReader(file)
        for pg in range(len(reader.pages)):
            page = reader.pages[pg]
            tex = page.extract_text()
            for i in tex:
                if type(i)==str and i == '\n':
                    text += ' '
                elif type(i)==str:
                    text+=i
        file.close()
        
    textRaw = text
    textOriginal = text.split(' ')

    # Searching for email
    regex = r"([a-zA-Z0-9._-]+@[a-zA-Z0-9._-]+\.[a-zA-Z0-9_-]+\.[a-zA-Z0-9_-]+) | ([a-zA-Z0-9._-]+@[a-zA-Z0-9._-]+\.[a-zA-Z0-9_-]+)"
    matches = re.search(regex, text)
    canEmail = ''
    try:
        canEmail = str((matches.group()).strip())
    except:
        canEmail = "Not Found"
    emailList.append(canEmail)

    # Searching Names
    doc = nlp(text)
    name = ''
    for ent in doc.ents:
      if str(ent.label_) == "Name":
        name = str(ent.text)
    nameList.append(name)

    # Searching required keywords
    found_skills = ''
    for ent in doc.ents:
      if str(ent.label_) == "Skills":
        found_skills = str(ent.text)
    foundKeywords=''
    for ky in keylist:
        if found_skills.find(ky) != -1:
            keyfound += 1
            foundKeywords += ky +', '
    foundKeywordList.append(foundKeywords)
    fileScore.append(keyfound*100/keylen)

In [ ]:
!pip install pandas --upgrade

In [ ]:
# Making a dataframe and exporting it to a csv
os.chdir(pathparent)
df = pd.DataFrame({"File Names" : fileList, r"% Keywords Matched" : fileScore,"Keywords Found":foundKeywordList,"Name":nameList,"Email":emailList})
df=df[df[r"% Keywords Matched"]!=0]
df = df.sort_values(r"% Keywords Matched",ascending=False)
print(df)
df.to_csv("/content/spacyResumeParcer/resumeResult.csv", index=False)
print("---  %s files scanned ---" % (fileLen))

## Testing a single Resume

In [ ]:

text = ''
fileName = '/content/inputFolder/2.pdf'
try:
    resume = docx2txt.process(fileName)
    for i in resume:
            if i == '\n':
                text += ' '
            else:
                text+=i
except:
    file = open(fileName,'rb')
    reader=PyPDF2.PdfReader(file)
    for pg in range(len(reader.pages)):
        page = reader.pages[pg]
        tex = page.extract_text()
        for i in tex:
            if type(i)==str and i == '\n':
                text += ' '
            elif type(i)==str:
                text+=i
    file.close()
text

In [ ]:
# Displaying Texts and Labels of Entities
doc = nlp(text)
for ent in doc.ents:
  print(ent.text,"  :  ",ent.label_)